## Import packages

In [1]:
cd /Users/zoltan/Dropbox/Python

/Users/zoltan/Dropbox/Python


In [2]:
pwd

u'/Users/zoltan/Dropbox/Python'

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
import cline_analysis as ca
import pandas as pd
import seaborn as sns
import datetime
import os
from scipy.signal import medfilt
import functools
from scipy.optimize import bisect
from scipy import stats

sns.set_style("whitegrid")
sns.set_style("ticks")
%matplotlib qt
%config InlineBackend.figure_format = 'svg'
plt.matplotlib.rcParams['svg.fonttype'] = 'svgfont' # fonts will be recognized by Adobe Illustrator

## Load data

In [5]:
dirname = '/Users/zoltan/Dropbox/Channels/Fluvial/Jurua/csv_files/'
fnames,clxs,clys,rbxs,lbxs,rbys,lbys,curvatures,ages,widths,dates = ca.load_data(dirname)

In [6]:
fnames

['Jurua_19871012.csv', 'Jurua_20170710.csv']

In [7]:
dates

[datetime.datetime(1987, 10, 12, 0, 0), datetime.datetime(2017, 7, 10, 0, 0)]

## Get migration rate

In [8]:
ts1 = 0 # first timestep
ts2 = 1 # second timestep

d = dates[ts2]-dates[ts1]
years = d.days/365.0

x = np.array(clxs[ts1])
y = np.array(clys[ts1])

xn = np.array(clxs[ts2])
yn = np.array(clys[ts2])

migr_rate, migr_sign, p, q = ca.get_migr_rate(x,y,xn,yn,years,0)

In [8]:
migr_rate = medfilt(savgol_filter(migr_rate,11,3),kernel_size=5) # smoothing
curv,s = ca.compute_curvature(x,y)
curv = medfilt(savgol_filter(curv,71,3),kernel_size=5) # smoothing

In [9]:
# set intervals affected by cu=toffs to NaN - specific to Jurua river segment
migr_rate[224:377] = np.NaN
migr_rate[2447:3121] = np.NaN
migr_rate[19132:19678] = np.NaN

In [10]:
plt.figure()
plt.plot(migr_rate)

In [10]:
from shapely.geometry import LineString
import geopandas as gpd

def write_shapefile(x,y,epsg,dirname,filename):
    coords=[]
    for i in range(len(x)):
        coords.append((x[i],y[i]))
    ls = LineString(coords)
    ls = gpd.GeoSeries(ls)
    ls.crs = {'init' :'epsg:'+str(epsg)}
    ls.to_file(dirname+filename+'.shp')
    
dirname = '/Users/zoltan/Dropbox/Channels/Fluvial/Curvature_paper_II/GIS_data/Jurua/'
epsg = 32619

year = str(dates[ts1].year)
day = str(dates[ts1].day)
if len(day)==1:
    day = '0'+day   
month = str(dates[ts1].month)
if len(month)==1:
    month = '0'+month
filename = year+month+day+'_cl'
write_shapefile(x,y,epsg=epsg,dirname=dirname,filename=filename)
filename = year+month+day+'_lb'
write_shapefile(lbxs[ts1],lbys[ts1],epsg=epsg,dirname=dirname,filename=filename)
filename = year+month+day+'_rb'
write_shapefile(rbxs[ts1],rbys[ts1],epsg=epsg,dirname=dirname,filename=filename)

year = str(dates[ts2].year)
day = str(dates[ts2].day)
if len(day)==1:
    day = '0'+day   
month = str(dates[ts2].month)
if len(month)==1:
    month = '0'+month
filename = year+month+day+'_cl'
write_shapefile(xn,yn,epsg=epsg,dirname=dirname,filename=filename)
filename = year+month+day+'_lb'
write_shapefile(lbxs[ts2],lbys[ts2],epsg=epsg,dirname=dirname,filename=filename)
filename = year+month+day+'_rb'
write_shapefile(rbxs[ts2],rbys[ts2],epsg=epsg,dirname=dirname,filename=filename)

## Read 'valid' inflection points and corresponding points of zero migration from CSV file

In [11]:
df = pd.read_csv('Jurua_LC08_L1TP_002064_20170710_20170725_01_T1_inflection_and_zero_migration_indices.csv')
LZC = np.array(df['index of inflection point'])
LZM = np.array(df['index of zero migration'])

In [12]:
# indices of bends affected by low erodibility and cutoffs (these have been picked manually)
erodibility_inds = [17,55,73,94,102,106,108,110,112,125,131,133]
cutoff_inds = [7,8,15,16,35,36,42,43,44,45,46,47,64,65,79,80,87,88,92,93,100,101,121,122]

## Plot curvature and migration rate series side-by-side

In [34]:
# plot curvature and migration rate along the channel

W = np.nanmean(widths[0]) # mean channel width

fig, ax1 = plt.subplots(figsize=(25,4))
plt.tight_layout()

y1 = 0.5
y2 = 0.0
y3 = -0.55
y4 = -1.0

for i in range(0,len(LZC)-1,2):
    xcoords = [s[LZC[i]],s[LZC[i+1]],s[LZC[i+1]],s[LZM[i+1]],s[LZM[i+1]],s[LZM[i]],s[LZM[i]],s[LZC[i]]]
    ycoords = [y1,y1,y2,y3,y4,y4,y3,y2]
    ax1.fill(xcoords,ycoords,color=[0.85,0.85,0.85],zorder=0)

deltas = 25.0
ax1.fill_between(s, 0, curv*W)
ax2 = ax1.twinx()
ax2.fill_between(s, 0, migr_rate, facecolor='green')

ax1.plot([0,max(s)],[0,0],'k--')
ax2.plot([0,max(s)],[0,0],'k--')

ax1.set_ylim(y4,y1)
ax2.set_ylim(-15,35)
ax1.set_xlim(0,s[-1])

for i in erodibility_inds:
    xcoords = [s[LZC[i]],s[LZC[i+1]],s[LZC[i+1]],s[LZM[i+1]],s[LZM[i+1]],s[LZM[i]],s[LZM[i]],s[LZC[i]]]
    ycoords = [y1,y1,y2,y3,y4,y4,y3,y2]
    ax1.fill(xcoords,ycoords,color=[1.0,0.85,0.85],zorder=0) 
    
for i in cutoff_inds:
    xcoords = [s[LZC[i]],s[LZC[i+1]],s[LZC[i+1]],s[LZM[i+1]],s[LZM[i+1]],s[LZM[i]],s[LZM[i]],s[LZC[i]]]
    ycoords = [y1,y1,y2,y3,y4,y4,y3,y2]
    ax1.fill(xcoords,ycoords,color=[0.85,1.0,0.85],zorder=0) 
    
for i in range(len(LZC)-1):
    if np.sum(np.isnan(migr_rate[LZM[i]:LZM[i+1]]))>0:
        xcoords = [s[LZC[i]],s[LZC[i+1]],s[LZC[i+1]],s[LZM[i+1]],s[LZM[i+1]],s[LZM[i]],s[LZM[i]],s[LZC[i]]]
        ycoords = [y1,y1,y2,y3,y4,y4,y3,y2]
        ax1.fill(xcoords,ycoords,color='w') 
        
for i in range(len(LZC)-1):
    if np.sum(np.isnan(migr_rate[LZM[i]:LZM[i+1]]))>0:
        xcoords = [s[LZC[i]],s[LZC[i+1]],s[LZC[i+1]],s[LZM[i+1]],s[LZM[i+1]],s[LZM[i]],s[LZM[i]],s[LZC[i]]]
        ycoords = [35,35,20.7145,0,-15,-15,0,20.7145]
        ax2.fill(xcoords,ycoords,color='w') 

for i in range(0,len(LZC)-1,2):
    ax1.text(s[LZC[i]],0.5,str(i),fontsize=12)

## Estimate lag between curvature and migration rate

In [17]:
window_length = 500
time_shifts = ca.get_time_shifts(migr_rate,curv,window_length)

In [18]:
plt.figure()
plt.plot(time_shifts)

In [19]:
# average lag
25.0*np.round(np.mean(time_shifts))

-1000.0

In [20]:
# average lag estimated from distances between inflection points and points of zero migration 
# (this is what was used in the paper)
np.mean(25.0*(LZM-LZC))

1072.6510067114093

## Estimate friction factor Cf

In [21]:
# first we need a continuous channel segment (e.g., no NaNs due to cutoffs)
q=np.array(q)
p=np.array(p)
         
i1 = 3121
i2 = 19132
i1n = p[np.where(q==i1)[0][0]]
i2n = p[np.where(q==i2)[0][0]]

xt = x[i1:i2]
yt = y[i1:i2]
xnt = xn[i1n:i2n]
ynt = yn[i1n:i2n]

plt.figure()
plt.plot(xt,yt)
plt.plot(xnt,ynt)
plt.axis('equal')

migr_rate_t, migr_sign_t, pt, qt = ca.get_migr_rate(xt,yt,xnt,ynt,years,0)

plt.figure()
plt.plot(migr_rate_t)

In [22]:
# this might take a while to run
kl = 20.0 # preliminary kl value (guesstimate)
k = 1
D = (W/18.8)**0.7092 # depth in meters (from width)

dx,dy,ds,s = ca.compute_derivatives(xt,yt)
curv_t, s = ca.compute_curvature(xt,yt)
curv_t = medfilt(savgol_filter(curv_t,71,3),kernel_size=5) # smoothing

migr_rate_t = medfilt(savgol_filter(migr_rate_t,71,3),kernel_size=5)

get_friction_factor_1 = functools.partial(ca.get_friction_factor,curvature=curv_t,migr_rate=migr_rate_t,
                                          kl=kl,W=W, k=k, D=D, s=s)

Cf_opt = bisect(get_friction_factor_1, 0.0002, 0.1)
print Cf_opt

0.002733984375


In [23]:
Cf_opt = 0.002733984375

## Estimate migration rate constant kl

In [25]:
# minimize the error between actual and predicted migration rates (using the 75th percentile)
errors = []
curv_t, s = ca.compute_curvature(xt,yt)
for i in np.arange(10,30):
    print i
    R1 = ca.get_predicted_migr_rate(curv_t,W=W,k=1,Cf=Cf_opt,D=D,kl=i,s=s)
    errors.append(np.abs(np.percentile(np.abs(R1),75)-np.percentile(np.abs(migr_rate_t[1:-1]),75)))
    
plt.figure()
plt.plot(np.arange(10,30),errors);

10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [26]:
kl_opt = 20.0 # the error is at minimum for kl = 26.0

## Plot actual migration rate against nominal migration rate

In [28]:
# kernel density and scatterplot of actual vs. nominal migration rate
w = np.nanmedian(widths[0]) 
curv_nodim = w*curv*kl_opt
lag = 40
plt.figure(figsize=(8,8))
sns.kdeplot(curv_nodim[:-lag][np.isnan(migr_rate[lag:])==0], migr_rate[lag:][np.isnan(migr_rate[lag:])==0],
           n_levels=20,shade=True,cmap='Blues',shade_lowest=False)
plt.scatter(curv_nodim[:-lag][::20],migr_rate[lag:][::20],c='k')
max_x = 10
plt.xlim(-max_x,max_x)
plt.ylim(-max_x,max_x)
plt.plot([-max_x,max_x],[-max_x,max_x],'k--')
plt.xlabel('nominal migration rate (m/year)', fontsize=14)
plt.ylabel('actual migration rate (m/year)', fontsize=14)

In [29]:
# get correlation coefficient for relationship between curvature and migration rate
slope, intercept, r_value, p_value, slope_std_rror = stats.linregress(curv_nodim[:-lag][np.isnan(migr_rate[lag:])==0],
                                                                      migr_rate[lag:][np.isnan(migr_rate[lag:])==0])
print r_value
print r_value**2
print p_value

0.72897056164
0.531398079738
0.0


In [30]:
# number of data points used in analysis
len(curv_nodim[:-lag][np.isnan(migr_rate[lag:])==0])

19257

In [31]:
# compute predicted migration rates
D = (w/18.8)**0.7092 # depth in meters (from width)
dx,dy,ds,s = ca.compute_derivatives(x,y)
R1 = ca.get_predicted_migr_rate(curv,W=w,k=1,Cf=Cf_opt,D=D,kl=kl_opt,s=s)

In [46]:
# plot actual and predicted migration rates
plt.figure()
plt.plot(s,migr_rate)
plt.plot(s,R1,'r')

In [32]:
# get correlation coefficient for relationship between actual and predicted migration rate
m_nonan = migr_rate[(np.isnan(R1)==0)&(np.isnan(migr_rate)==0)]
R_nonan = R1[(np.isnan(R1)==0)&(np.isnan(migr_rate)==0)]

slope, intercept, r_value, p_value, slope_std_rror = stats.linregress(R_nonan,m_nonan)
print r_value
print r_value**2
print p_value

0.756121037172
0.571719022854
0.0


In [33]:
# 90th percentile of migration rate
np.percentile(np.abs(m_nonan),90)

9.5589875078247175

In [36]:
# plot actual vs. predicted migration rate
max_m = 10
plt.figure(figsize=(8,8))
sns.kdeplot(R_nonan,m_nonan,n_levels=10,shade=True,cmap='Blues',shade_lowest=False)
plt.plot([-max_m,max_m],[-max_m,max_m],'k--') 
plt.scatter(R_nonan[::20],m_nonan[::20],c='k')
plt.xlim(-max_m,max_m)
plt.ylim(-max_m,max_m)
plt.xlabel('predicted migration rate (m/year)', fontsize=14)
plt.ylabel('actual migration rate (m/year)', fontsize=14)

In [51]:
# add points affected by cutoffs and low erodibility
for i in erodibility_inds:
    plt.scatter(R1[LZC[i]:LZC[i+1]][::10],migr_rate[LZC[i]:LZC[i+1]][::10],c='r')
for i in cutoff_inds:
    plt.scatter(R1[LZC[i]:LZC[i+1]][::10],migr_rate[LZC[i]:LZC[i+1]][::10],c='g')